In [10]:
import requests
from flask import Flask, request, jsonify, Response, make_response
import json
from filelock import Timeout, FileLock
import email, smtplib, ssl
from email.mime.text import MIMEText
import urllib.parse

import smtplib
import threading
from email.mime.text import MIMEText

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
import requests
from time import sleep
import time
import logging
import json
from filelock import Timeout, FileLock
from apscheduler.schedulers.background import BackgroundScheduler
from concurrent.futures import ThreadPoolExecutor
import sys

In [11]:
GATEWAYS_FILE_PATH = "./data/gateways.json"
PLAYER_DATA_FILE_PATH = "./data/player_data.json"
PLAYER_DATA_LOCK_FILE_PATH = "./data/player_data.json.lock"
#Do not use gmail as phone carriers typically block all gmail emails automatically
#Zoho seems to work well
sender_credentials = ('email', 'password')

In [12]:
countries = set()
providers = set()
with open(GATEWAYS_FILE_PATH) as gateways_file:
    gateways = json.load(gateways_file)
    for g in gateways:
        countries.add(g.get("country").upper())
        providers.add(g.get("provider").upper())

In [13]:
def text_registered(country, provider, number, username):
    sender_email, email_password = sender_credentials
    link_found = False
    link = ''
    for g in gateways:
        if g.get("country").upper() == country.upper() and g.get("provider").upper() == provider.upper():
            link_found = True
            link = g.get("link")

    link_split = link.split("#")
    receiver_email = link_split[0] + str(number) + link_split[-1]

    games_link = f"https://www.gpstats.dev/textme/RemoveMyNumber?number={number}&username={urllib.parse.quote_plus(username)}"

    body_1 = f"Your {username} bgs account is registered to receive turn reminders at this number :)"
    body_2 = games_link

    msg1 = MIMEText(body_1, 'plain')
    msg1['Subject'] = "Congratulations!"
    msg1['From'] = sender_email
    msg1['To'] = receiver_email
    msg2 = MIMEText(body_2, 'plain')
    msg2['Subject'] = "Remove me"
    msg2['From'] = sender_email
    msg2['To'] = receiver_email

    server = smtplib.SMTP_SSL('smtppro.zoho.com', 465)

    try:
        server.login(sender_email, email_password)
        server.sendmail(sender_email, [receiver_email], msg1.as_string())
        server.sendmail(sender_email, [receiver_email], msg2.as_string())
    finally:
        server.quit()

    return link_found

In [14]:
def text_removed(country, provider, number, username):
    sender_email, email_password = sender_credentials
    link_found = False
    link = ''
    for g in gateways:
        if g.get("country").upper() == country.upper() and g.get("provider").upper() == provider.upper():
            link_found = True
            link = g.get("link")

    link_split = link.split("#")
    receiver_email = link_split[0] + str(number) + link_split[-1]

    games_link = f"https://www.gpstats.dev/textme/AddMyNumber?country={urllib.parse.quote_plus(country)}&provider={urllib.parse.quote_plus(provider)}&number={number}&username={urllib.parse.quote_plus(username)}"

    body_1 = f"Your {username} bgs account has been removed"
    body_2 = games_link

    msg1 = MIMEText(body_1, 'plain')
    msg1['Subject'] = "Goodbye!"
    msg1['From'] = sender_email
    msg1['To'] = receiver_email
    msg2 = MIMEText(body_2, 'plain')
    msg2['Subject'] = "Add me back"
    msg2['From'] = sender_email
    msg2['To'] = receiver_email

    server = smtplib.SMTP_SSL('smtppro.zoho.com', 465)

    try:
        server.login(sender_email, email_password)
        server.sendmail(sender_email, [receiver_email], msg1.as_string())
        server.sendmail(sender_email, [receiver_email], msg2.as_string())
    finally:
        server.quit()

    return link_found

Example of Simple Flask Rest Endpoint to test texting service

In [15]:
def _build_cors_preflight_response():
    response = make_response()
    response.headers.add("Access-Control-Allow-Origin", "*")
    response.headers.add('Access-Control-Allow-Headers', "*")
    response.headers.add('Access-Control-Allow-Methods', "*")
    return response


def _corsify_actual_response(response):
    response.headers.add("Access-Control-Allow-Origin", "*")
    return response


app = Flask(__name__)


@app.route("/")
def hello():
    return "TextMe Running"


@app.route("/AddMyNumber", methods=['GET', 'PUT'])
def add_number():
    if request.method == "OPTIONS":
        return _build_cors_preflight_response()
    else:
        player_data = dict()

        args_dict = request.args.to_dict()
        country = args_dict.get("country")
        if country.upper() not in countries:
            return _corsify_actual_response(jsonify("{'result': 'country_not_in_list'}")), 403
        provider = args_dict.get("provider")
        if provider.upper() not in providers:
            return _corsify_actual_response(jsonify("{'result': 'provider_not_in_list'}")), 403
        number = args_dict.get("number")
        if not number.isdigit():
            return _corsify_actual_response(jsonify("{'result': 'number_has_nondigit_chars'}")), 403
        username = args_dict.get("username").lower()

        lock = FileLock(PLAYER_DATA_LOCK_FILE_PATH)
        with lock:
            with open(PLAYER_DATA_FILE_PATH) as player_data_file:
                player_data = json.load(player_data_file)

        if username in player_data:
            player_data.pop(username)

        new_data = {
            "username": username,
            "country": country,
            "provider": provider,
            "number": number
        }
        player_data[username] = new_data
        with lock:
            with open(PLAYER_DATA_FILE_PATH, "w") as player_data_file:
                json.dump(player_data, player_data_file, indent=4, separators=(',', ': '))
                try:
                    text_registered(country, provider, number, username)
                except:
                    pass
                return _corsify_actual_response(jsonify("{'result': 'player_added'}")), 200


@app.route("/RemoveMyNumber", methods=['GET', 'DELETE'])
def remove_number():
    if request.method == "OPTIONS":
        return _build_cors_preflight_response()
    else:
        args_dict = request.args.to_dict()

        number = args_dict.get("number")
        if not number.isdigit():
            return _corsify_actual_response(jsonify("{'result': 'number_has_nondigit_chars'}")), 403
        username = args_dict.get("username").lower()

        lock = FileLock(PLAYER_DATA_LOCK_FILE_PATH)
        with lock:
            with open(PLAYER_DATA_FILE_PATH) as player_data_file:
                player_data = json.load(player_data_file)

        cur_player_data = None
        if username in player_data and str(player_data.get(username).get('number')) == str(number):
            cur_player_data = player_data.get(username)
            player_data.pop(username)

        with lock:
            with open(PLAYER_DATA_FILE_PATH, "w") as player_data_file:
                json.dump(player_data, player_data_file, indent=4, separators=(',', ': '))
                try:
                    if cur_player_data:
                        text_removed(cur_player_data.get('country'), cur_player_data.get('provider'),
                                     cur_player_data.get('number'), username)
                except:
                    pass
                return _corsify_actual_response(jsonify("{'result': 'player_removed'}")), 200


@app.route("/CheckUsername", methods=['GET', 'OPTIONS'])
def check_username():
    if request.method == "OPTIONS":
        return _build_cors_preflight_response()
    else:
        args_dict = request.args.to_dict()
        username = args_dict.get("username")

        resp = requests.get(
            url='https://www.boardgamers.space/user/' + username)

        return _corsify_actual_response(jsonify(resp.ok)), resp.status_code


if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)


Example of threaded texting service

In [16]:
class Texter:
    def __init__(self):

        self.sender_credentials = sender_credentials
        with open(GATEWAYS_FILE_PATH) as gateways_file:
            self.gateways = json.load(gateways_file)

        # Thread lock for async-updated data
        self.player_data_lock = threading.Lock()
        self.active_games_lock = threading.Lock()
        self.game_state_lock = threading.Lock()

        # File lock because rest endpoint updating this file also
        player_data_file_lock = FileLock(PLAYER_DATA_LOCK_FILE_PATH)
        with player_data_file_lock:
            with open(PLAYER_DATA_FILE_PATH) as player_data_file:
                self.player_data = json.load(player_data_file)

        # no files for simplicity, will just text again if your server crashes and gets restarted (not automatic)
        self.active_games = set()
        self.game_state = dict()

        # install webdriver on init
        self.op = webdriver.ChromeOptions()
        self.op.add_argument('--headless')
        self.op.add_argument('--no-sandbox')
        self.op.add_argument('--disable-dev-shm-usage')
        self.op.add_argument("--window-size=1920,1080")
        self.op.add_argument("--start-maximized")
        self.op.add_argument("--disable-gpu")
        self.op.add_argument('--disable-extensions')
        self.driver = webdriver.Chrome(ChromeDriverManager("100.0.4896.60", chrome_type=ChromeType.CHROMIUM).install(),
                                       options=self.op)

        # Thread pool for texting (can be slow)
        self.updating_executor = ThreadPoolExecutor(max_workers=20)
        self.texting_executor = ThreadPoolExecutor(max_workers=20)

    def __del__(self):
        self.driver.close()

    def update_player_data(self):
        player_data_file_lock = FileLock(PLAYER_DATA_LOCK_FILE_PATH)
        with player_data_file_lock:
            with open(PLAYER_DATA_FILE_PATH) as player_data_file:
                self.player_data_lock.acquire()
                try:
                    self.player_data = json.load(player_data_file)
                    print(time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()) + " : updated player data")
                finally:
                    self.player_data_lock.release()

    def update_active_games(self):
        url_games = 'https://www.boardgamers.space/boardgame/gaia-project/games'

        try:
            self.driver.get(url=url_games)
        except:
            self.op = webdriver.ChromeOptions()
            self.op.add_argument('--headless')
            self.op.add_argument('--no-sandbox')
            self.op.add_argument('--disable-dev-shm-usage')
            self.op.add_argument("--window-size=1920,1080")
            self.op.add_argument("--start-maximized")
            self.op.add_argument("--disable-gpu")
            self.op.add_argument('--disable-extensions')
            self.driver = webdriver.Chrome(
                ChromeDriverManager("100.0.4896.60", chrome_type=ChromeType.CHROMIUM).install(),
                options=self.op)
            self.driver.get(url=url_games)
        print(time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()) + " : updating active games")

        sleep(.5)
        while self.driver.execute_script("return document.readyState") != "complete":
            sleep(.5)

        num_games_found = int(self.driver.find_elements_by_class_name("card-title")[1].text.split('(')[1].split(')')[0])

        # link to click for next page
        next_button = self.driver.find_elements_by_class_name("page-link")[-2]

        # list of names items
        active_games_list = self.driver.find_elements_by_class_name("game-list")[-1]
        game_names = active_games_list.find_elements_by_class_name("game-name")

        self.active_games_lock.acquire()
        try:
            for name in game_names:
                n = name.text
                if n not in self.active_games:
                    self.active_games.add(n)
        finally:
            self.active_games_lock.release()

        for i in range(num_games_found // 10):
            try:
                next_button.click()
                sleep(.5)
            except:
                continue

            # wait for JS to load -> elements are same, only text and class are updated
            while self.driver.execute_script("return document.readyState") != "complete":
                sleep(.5)

            next_button = self.driver.find_elements_by_class_name("page-link")[-2]

            active_games_list = self.driver.find_elements_by_class_name("game-list")[-1]
            game_names = active_games_list.find_elements_by_class_name("game-name")

            self.active_games_lock.acquire()
            try:
                for name in game_names:
                    n = name.text
                    if n not in self.active_games:
                        self.active_games.add(n)
            finally:
                self.active_games_lock.release()

    def text(self, country, provider, number, username, game):
        sender_email, email_password = self.sender_credentials
        link_found = False
        link = ''
        for g in self.gateways:
            if g.get("country").upper() == country.upper() and g.get("provider").upper() == provider.upper():
                link_found = True
                link = g.get("link")

        link_split = link.split("#")
        receiver_email = link_split[0] + str(number) + link_split[-1]

        games_link = "https://www.boardgamers.space/game/" + game

        body_1 = f"{game}"
        body_2 = f"{games_link}"

        msg1 = MIMEText(body_1, 'plain')
        msg1['Subject'] = f"{username}'s move"
        msg1['From'] = sender_email
        msg1['To'] = receiver_email
        msg2 = MIMEText(body_2, 'plain')
        msg2['Subject'] = "Goto"
        msg2['From'] = sender_email
        msg2['To'] = receiver_email

        server = smtplib.SMTP_SSL('smtppro.zoho.com', 465)

        try:
            server.login(sender_email, email_password)
            server.sendmail(sender_email, [receiver_email], msg1.as_string())
            server.sendmail(sender_email, [receiver_email], msg2.as_string())
        finally:
            server.quit()

        return link_found

    def check_games_and_update(self):
        base_url = "https://www.boardgamers.space/api/game/"

        self.active_games_lock.acquire()
        try:
            games = self.active_games.copy()
        finally:
            self.active_games_lock.release()

        for game_name in games:
            self.updating_executor.submit(self.check_game_and_update, game_name)

    def check_game_and_update(self, game_name):
        base_url = "https://www.boardgamers.space/api/game/"

        with requests.Session() as session:
            with session.get(base_url + game_name) as response:
                if response.status_code != 200:
                    return

                game_tree = response.json()

                if game_tree['status'] != "active":
                    # remove from active games
                    self.active_games_lock.acquire()
                    try:
                        self.active_games.remove(game_name)
                    finally:
                        self.active_games_lock.release()

                    # remove from game state
                    self.game_state_lock.acquire()
                    try:
                        self.game_state.pop(game_name)
                    finally:
                        self.game_state_lock.release()
                else:
                    if game_name in self.game_state:
                        cur_player_id = self.game_state.get(game_name)
                        moving_player_id = game_tree['currentPlayers'][0]['_id']
                        if moving_player_id == cur_player_id:
                            return
                        else:
                            self.game_state_lock.acquire()
                            try:
                                self.game_state[game_name] = moving_player_id
                            finally:
                                self.game_state_lock.release()

                            for player in game_tree['players']:
                                if player['_id'] == moving_player_id:
                                    username = str(player['name']).lower()
                                    if username in self.player_data:
                                        user_data = self.player_data.get(username)
                                        print(time.strftime("%Y-%m-%d %H:%M:%S",
                                                            time.gmtime()) + " : sending text : " + username + " " + user_data.get(
                                            'country') + " " + user_data.get('provider') + " " + user_data.get(
                                            'number'))
                                        self.texting_executor.submit(self.text, user_data.get('country'),
                                                                     user_data.get('provider'), user_data.get('number'),
                                                                     username, game_name)
                    else:
                        moving_player_id = game_tree['currentPlayers'][0]['_id']

                        self.game_state_lock.acquire()
                        try:
                            self.game_state[game_name] = moving_player_id
                        finally:
                            self.game_state_lock.release()

                        for player in game_tree['players']:
                            if player['_id'] == moving_player_id:
                                username = str(player['name']).lower()
                                if username in self.player_data:
                                    user_data = self.player_data.get(username)
                                    print(time.strftime("%Y-%m-%d %H:%M:%S",
                                                        time.gmtime()) + " : sending text : " + username + " " + user_data.get(
                                        'country') + " " + user_data.get('provider') + " " + user_data.get('number'))
                                    self.texting_executor.submit(self.text, user_data.get('country'),
                                                                 user_data.get('provider'), user_data.get('number'),
                                                                 username, game_name)

In [ ]:
texter = Texter()
texter.update_player_data()
texter.update_active_games()
texter.check_games_and_update()

"""
    sched = BackgroundScheduler(daemon=False)
    sched.add_job(texter.update_player_data, 'interval', seconds=10)
    sched.add_job(texter.update_active_games, 'interval', seconds=60)
    sched.add_job(texter.check_games_and_update, 'interval', seconds=30)
    sched.start()
"""